In [1]:
import pandas as pd

In [2]:
import requests
import json

def get_pushshift_data(data_type, **kwargs):
    """
    Gets data from the pushshift api.
 
    data_type can be 'comment' or 'submission'
    The rest of the args are interpreted as payload.
 
    Read more: https://github.com/pushshift/api
    """
 
    base_url = f"https://files.pushshift.io/reddit/comments/"
    payload = kwargs
    request = requests.get(base_url, params=payload)
    return request.json()

In [18]:
from bs4 import BeautifulSoup

import httplib
httplib.HTTPConnection._http_vsn = 10
httplib.HTTPConnection._http_vsn_str = 'HTTP/1.0'

prefix = 'c'
pattern_filter = "*2018*&*2019*"
download_filter = list([x.strip() for x in pattern_filter.split('&')])
download_filter.sort()

keyword_filter = ''
present_download_files = 'Downloaded->           '       
keyword_filter = 'rc'
present_download_files = 'RC ' + present_download_files
r_df  = requests.get("https://files.pushshift.io/reddit/comments/")
data_df = r_df.text
soup_df = BeautifulSoup(data_df,features="lxml")

#Get the href links from the website
list_of_href_df = set()
for link_df in soup_df.find_all('a'):
    href_link_df = link_df.get('href')
    list_of_href_df.add("https://files.pushshift.io/reddit/comments/" + href_link_df[2:])

#Clean from external href links     
clean_list_of_href_df = []
for loh_df in list_of_href_df:
    if keyword_filter in loh_df.lower():
        clean_list_of_href_df.append(loh_df)
        
#Filter the links
matching_fnmatch_list = []
if pattern_filter != '':
    if prefix == '*':
        pass
    for dfilter in download_filter:
        fnmatch_list = fnmatch.filter(clean_list_of_href_df, dfilter) 
        i = 0
        for fnl in fnmatch_list:
            '''temporary break to test'''
            if i == 2:
                break
            i += 1
            matching_fnmatch_list.append(fnl)

ModuleNotFoundError: No module named 'httplib'

In [15]:
matching_fnmatch_list

['https://files.pushshift.io/reddit/comments/RC_2018-10.xz',
 'https://files.pushshift.io/reddit/comments/RC_2018-12.zst',
 'https://files.pushshift.io/reddit/comments/RC_2019-10.zst',
 'https://files.pushshift.io/reddit/comments/RC_2019-05.zst']

In [ ]:
'https://files.pushshift.io/reddit/comments/RC_2019-01.zst',
'https://files.pushshift.io/reddit/comments/RC_2019-02.zst',
'https://files.pushshift.io/reddit/comments/RC_2019-03.zst',
'https://files.pushshift.io/reddit/comments/RC_2019-04.zst',
'https://files.pushshift.io/reddit/comments/RC_2019-05.zst',
'https://files.pushshift.io/reddit/comments/RC_2019-06.zst',
'https://files.pushshift.io/reddit/comments/RC_2019-07.zst',
'https://files.pushshift.io/reddit/comments/RC_2019-08.zst',
'https://files.pushshift.io/reddit/comments/RC_2019-09.zst',
'https://files.pushshift.io/reddit/comments/RC_2019-10.zst',
'https://files.pushshift.io/reddit/comments/RC_2019-11.zst',
'https://files.pushshift.io/reddit/comments/RC_2019-12.zst'

In [17]:
path_to_save_filename = 'C:/Users/Donghyeok/Desktop/Work Folder/Reddit Sarcasm Detection/reddit_pushshift-master/'
request = urllib.request.Request('https://files.pushshift.io/reddit/comments/RC_2019-12.zst')
response = urllib.request.urlopen(request)
data_content = response.read()
with open(path_to_save_filename, 'wb') as wf:    
    wf.write(data_content)                 
    print('Downloaded    ->    ' + filename)

IncompleteRead: IncompleteRead(507575 bytes read, 15907980594 more expected)

In [ ]:
httplib.HTTPConnection._http_vsn = 11
httplib.HTTPConnection._http_vsn_str = 'HTTP/1.1'

In [3]:
import os
import time
import sys
import fnmatch
import requests
import urllib.request
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool as ThreadPool

import bz2
import lzma
import gzip
import shutil
from pathlib import Path
import zstandard as zstd
import multiprocessing as mp

class reddit:
    def multithread_download_files_func(self,list_of_file):
        filename = list_of_file[list_of_file.rfind("/")+1:]
        path_to_save_filename = self.ptsf_download_files + filename
        if not os.path.exists(path_to_save_filename): 
            data_content = None
            try:
                request = urllib.request.Request(list_of_file)
                response = urllib.request.urlopen(request)
                data_content = response.read()
            except urllib.error.HTTPError:
                retries = 1
                success = False
                while not success:
                    try:
                        response = urllib.request.urlopen(list_of_file)
                        data_content = response.read()                        
                        success = True
                    except Exception:
                        wait = retries * 15;
                        print('Service Temporarily Unavailable! Retrying in ' + str(wait) + 's on file: ' + filename)
                        sys.stdout.flush()                          
                        time.sleep(wait)
                        retries += 1               
            if data_content:
                with open(path_to_save_filename, 'wb') as wf:    
                    wf.write(data_content)                 
                    print(self.present_download_files + filename)
    def download_files(self,filter_files_df,url_to_download_df,path_to_save_file_df,prefix):
        if filter_files_df == '*':
            reddit_years = [2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]
            filter_files_df = ''
            for idx, ry in enumerate(reddit_years):
                filter_files_df += '*' + str(ry) + '*'
                if (idx != len(reddit_years)-1):
                    filter_files_df += '&'   

        download_filter = list([x.strip() for x in filter_files_df.split('&')])
        download_filter.sort()

        keyword_filter = ''
        self.present_download_files = 'Downloaded->           '
        if prefix == 's':
            self.ptsf_download_files = path_to_save_file_df + 'Step 1 - Data Lake\Reddit Submissions\\'
            keyword_filter = 'rs'
            self.present_download_files = 'RS ' + self.present_download_files
            #Include 1 previous year for submissions only
            #min_year = download_filter[0] 
            #int_min_year = int(min_year[1:-1])-1 #minus 1 year            
            #download_filter.append('*' + str(int_min_year) + '*')  
            #download_filter.sort()

        if prefix == 'c':
            self.ptsf_download_files = path_to_save_file_df + 'Step 1 - Data Lake\Reddit Comments\\'        
            keyword_filter = 'rc'
            self.present_download_files = 'RC ' + self.present_download_files

        #If folder doesn't exist, create one
        if not os.path.exists(os.path.dirname(self.ptsf_download_files)):
            os.makedirs(os.path.dirname(self.ptsf_download_files))        

        r_df  = requests.get(url_to_download_df)
        data_df = r_df.text
        soup_df = BeautifulSoup(data_df,features="lxml")

        #Get the href links from the website
        list_of_href_df = set()
        for link_df in soup_df.find_all('a'):
            href_link_df = link_df.get('href')
            list_of_href_df.add(url_to_download_df + href_link_df[2:])

        #Clean from external href links     
        clean_list_of_href_df = []
        for loh_df in list_of_href_df:
            if keyword_filter in loh_df.lower():
                clean_list_of_href_df.append(loh_df)

        #Filter the links
        matching_fnmatch_list = []
        if filter_files_df != '':
            if prefix == '*':
                pass
            for dfilter in download_filter:
                fnmatch_list = fnmatch.filter(clean_list_of_href_df, dfilter) 
                i = 0
                for fnl in fnmatch_list:
                    '''temporary break to test'''
                    if i == 2:
                        break
                    i += 1
                    matching_fnmatch_list.append(fnl)

        if not matching_fnmatch_list:
            matching_fnmatch_list = clean_list_of_href_df.sort()
        matching_fnmatch_list.sort()

        p = ThreadPool(200)
        p.map(self.multithread_download_files_func, matching_fnmatch_list)

In [4]:
class decompress:
        
    def multiprocess_extaction_func(self,path_to_read_file):
         file_name = path_to_read_file[path_to_read_file.rfind("\\")+1:]
         file_name_extension = Path(file_name).suffix
         file_name_wo_extension = file_name[:file_name.rfind(file_name_extension)]
         path_to_save_file = self.path_to_save_file_df + file_name_wo_extension
         if file_name_extension == ".bz2":
             with bz2.BZ2File(path_to_read_file,"rb") as fr, open(path_to_save_file,"wb") as fw:
                 shutil.copyfileobj(fr,fw,length = 65536)
                 print(self.present_decompress_file + file_name_wo_extension)
         elif file_name_extension == ".xz":
             with lzma.open(path_to_read_file, 'rb') as fr, open(path_to_save_file, 'wb') as fw:
                 shutil.copyfileobj(fr, fw, 65536)
                 print(self.present_decompress_file + file_name_wo_extension)              
         elif file_name_extension == ".gz": 
             with gzip.open(path_to_read_file, 'rb') as fr, open(path_to_save_file, 'wb') as fw:
                 shutil.copyfileobj(fr, fw, 65536)
                 print(self.present_decompress_file + file_name_wo_extension)             
         elif file_name_extension == ".zst":
             dctx = zstd.ZstdDecompressor()
             with open(path_to_read_file, 'rb') as ifh, open(path_to_save_file, 'wb') as ofh:
                dctx.copy_stream(ifh, ofh, write_size=65536) 
                print(self.present_decompress_file + file_name_wo_extension)               
     
    def decompress_file(self,dir_to_read_file,prefix):
        self.present_decompress_file = 'Decompressed->         '  
        if prefix == 's':
            self.present_decompress_file = 'RS ' + self.present_decompress_file
            self.dtrf_df = dir_to_read_file + 'Step 1 - Data Lake\Reddit Submissions\\'
            self.path_to_save_file_df = dir_to_read_file + 'Step 2 - Decompress\Reddit Submissions\\'  
        if prefix == 'c':
            self.present_decompress_file = 'RC ' + self.present_decompress_file
            self.dtrf_df = dir_to_read_file + 'Step 1 - Data Lake\Reddit Comments\\'
            self.path_to_save_file_df = dir_to_read_file + 'Step 2 - Decompress\Reddit Comments\\'   
        if prefix == 'w':
            self.present_decompress_file = 'WK ' + self.present_decompress_file
            self.dtrf_df = dir_to_read_file + 'Step 1 - Data Lake\Wikipedia Pagecounts\\'
            self.path_to_save_file_df = dir_to_read_file + 'Step 2 - Decompress\Wikipedia Pagecounts\\'  
        extraction_list = []           
        directory = os.fsencode(self.dtrf_df)
        if not os.path.exists(os.path.dirname(self.path_to_save_file_df)):
            os.makedirs(os.path.dirname(self.path_to_save_file_df))                
        
        for file_to_uncompress in os.listdir(directory):
            extraction_list.append(directory.decode() + file_to_uncompress.decode())  

        extraction_list.sort()    
        with mp.Pool(mp.cpu_count()) as p:
            p.map(self.multiprocess_extaction_func, extraction_list)

In [5]:
pattern_filter = "*2019*"
reddit_repo = reddit()
dir_of_file = 'C:/Users/Donghyeok/Desktop/Work Folder/Reddit Sarcasm Detection/reddit_pushshift-master/'
print('---  Reddit Comments')
reddit_repo.download_files(pattern_filter,"https://files.pushshift.io/reddit/comments/",dir_of_file,'c')  

---  Reddit Comments


IncompleteRead: IncompleteRead(392196 bytes read, 14693529258 more expected)

In [ ]:
decompress_repo = decompress()
print('---  Reddit Comments')
decompress_repo.decompress_file(dir_of_file,'c')